In [1]:
import numpy as np
import itertools
from itertools import chain
import networkx as nx
import pandas as pd
from random import randint, randrange
import string
import os
from numpy import random
from numpy.linalg import matrix_rank
from sklearn.covariance import GraphicalLasso
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None


import warnings
warnings.filterwarnings('ignore')

from MLTCalculator import flatten, getGLCovariance, adjacencyJacobian, MLTBig


In [2]:
def makeMLTFrame(starta, stepa, enda, numpera, ndim, nsamp):
    data = []
    for a in np.arange(enda, starta, -stepa):
        sufficient = 0
        for i in range(numpera):
            counter = numpera
            #print(a)
            try:
                S = getGLCovariance(ndim, a, nsamp)
            except:
                counter -= 1

            G = nx.from_numpy_array(S)
            G.remove_edges_from(nx.selfloop_edges(G))
            #print(S)
            MLT = MLTBig(G)
            if MLT <= nsamp:
                sufficient += 1
        data.append([a, sufficient/counter, counter, nsamp])
    df = pd.DataFrame(data)
    df.columns = ["alpha", "pr", "samples", "nsamp"]
    return(df)

In [3]:
GLCOV = getGLCovariance(3,0.001,2)
print(GLCOV)
G = nx.from_numpy_array(GLCOV)
G.remove_edges_from(nx.selfloop_edges(G))

    
    
adjacencyJacobian(G)
MLTBig(G)

[[ 1.57648253  0.17437789  0.04292752]
 [ 0.17437789  0.04876076 -0.12964989]
 [ 0.04292752 -0.12964989  0.62925146]]


3

In [4]:
GLCOV = getGLCovariance(3,0.1,2)
print(GLCOV)
G = nx.from_numpy_array(GLCOV)
G.remove_edges_from(nx.selfloop_edges(G))

adjacencyJacobian(G)
print(MLTBig(G))

[[ 6.73209758  0.96157094 -0.33011763]
 [ 0.96157094  0.77912888 -0.75804627]
 [-0.33011763 -0.75804627  1.11176234]]
3


In [5]:
%%time
df_3_3_1000 = makeMLTFrame(0.01, 0.01, 1.0, 1000, 3, 3)

df_3_2_1000 = makeMLTFrame(0.01, 0.01, 1.0, 1000, 3, 2)

df_3d_data = pd.concat([df_3_3_1000, df_3_2_1000], ignore_index = True)

df_3d_data.head()

sns.scatterplot(df_3d_data, x = "alpha", y = "pr", hue = "nsamp", palette = "tab10").set(title = "Probability MLT <= n, 3 Dimensions, 1000 Trials")
plt.tight_layout()
#plt.savefig('3dPlot.png', dpi = 300)